In [1]:
import tensorflow as tf
import numpy as np
from PIL import Image
import pandas as pd
import cv2


input_dim = 5  # one-hot size
hidden_size = 4  # output from the LSTM. 5 to directly predict one-hot
batch_size = 50   # one sentence
sequence_length = 5  # |ihello| == 6
learning_rate = 0.0001
data_mean = 17.119790415671556
data_std = 63.36037302373175

In [2]:
X = tf.placeholder(tf.float32, [None, 200, 400, 1])
Y = tf.placeholder(tf.float32, [None, 4])

with tf.variable_scope("cnn"):
    
    # conv : 5, 200, 400, 16
    conv = tf.layers.conv2d(inputs=X, filters=16, kernel_size=[4, 8], padding='SAME', activation=tf.nn.relu)
    # pool : 5, 50, 99, 16
    pool = tf.layers.max_pooling2d(inputs=conv, pool_size=[2, 4], padding='SAME', strides=1)
    # conv2 
    conv2 = tf.layers.conv2d(inputs=pool, filters=32, kernel_size=[4, 8], padding='SAME', activation=tf.nn.relu)
    # pool2 
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[4, 8], padding='SAME', strides=2)
    # conv3
    conv3 = tf.layers.conv2d(inputs=pool2, filters=64, kernel_size=[4, 8], padding='SAME', activation=tf.nn.relu)
    # pool3 
    pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[4, 8], padding='SAME', strides=2)
    
    flat = tf.reshape(pool3, [-1, 50*100*64])
    w = tf.get_variable("w", [hidden_size, hidden_size])
    b = tf.get_variable("b", [hidden_size])
    
    logit = tf.layers.dense(inputs=flat, units=4)
    outputs = tf.sigmoid(tf.matmul(logit, w) + b)
    
    loss = -tf.reduce_mean(Y*tf.log(outputs)+(1-Y)*(tf.log(1-outputs)))
    train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

In [3]:
saver = tf.train.Saver()

with tf.Session() as sess:
    res = raw_input("Do you want addtional learning? (Name or Enter)")
    if res:
        saver.restore(sess, "./model/"+res+".ckpt")
    else:
        sess.run(tf.global_variables_initializer())
    model_name = raw_input("Model_Name: ")
    
    try:
        for epoch in range(5):
            for ims in range(1, 8):

                data_df = pd.read_csv('/mnt/d/imgs/img'+str(1)+'/log.csv', names=['name','key_out'])
                names = data_df['name'].values
                # keyouts = [8, 1, 4, 2, 15, ...]
                key_out = [eval(x) for x in data_df['key_out'].values]

                while len(names):
                    imgs = []
                    offset = min(batch_size, len(names))
                    batch_x = names[0:offset]
                    batch_y = key_out[0:offset]
                    names = names[offset:]
                    key_out = key_out[offset:]

                    for name in batch_x:
                        img = Image.open('/mnt/d/imgs/img'+str(1)+'/img/'+name)
                        data = np.array( img, dtype='uint8')
                        data = data.reshape([200,400,1])
                        data = (data-data_mean) / data_std
                        imgs.append(data)
                    sess.run(train, feed_dict={X:np.array(imgs, np.float32), Y:batch_y})
                    print sess.run(loss, feed_dict={X:np.array(imgs, np.float32), Y:batch_y})
            save_path = saver.save(sess, "./model/"+model_name+"_"+str(epoch+1)+".ckpt")
            print 'epoch', epoch+1, sess.run(loss, feed_dict={X:np.array(imgs, np.float32), Y:batch_y})
    finally:
        save_path = saver.save(sess, "./model/"+model_name+"_ex.ckpt")

Do you want addtional learning? (Name or Enter)
Model_Name: full3
1.1768651
0.9363733
0.7975978
0.62748283
0.7029389
0.8211725
0.70896006
0.6329964
0.6056325
0.6186061
0.6294707
0.57006294
0.62423384
0.56977576
0.5898482
0.5506858
0.5782595
0.56913537
0.5379408
0.57219404
0.48652032
0.5445446
0.4445079
0.5634377
0.47659984
0.4938228
0.46617344
0.37264624
0.55569595
0.34132537
0.49316773
0.3704924
0.54655725
0.5138206
0.5343366
0.47046697
0.4683155
0.43517205
0.48949513
0.45183945
0.45503867
0.4667549
0.4890105
0.689388
0.44718525
0.46119133
0.44679776
0.44119573
0.4322179
0.4597727
0.40224254
0.4126735
0.4601508
0.38780412
0.46804187
0.38332188
0.48441133
0.46508363
0.40459454
0.41893548
0.3682716
0.5225318
0.35337242
0.4425701
0.37942082
0.44880858
0.44707352
0.49879143
0.4535133
0.40273625
0.41484794
0.4814396
0.4277754
0.44551152
0.44453964
0.46966943
0.7268066
0.43015686
0.4460139
0.4319863
0.42626667
0.4078064
0.4479655
0.3549634
0.40219393
0.44079164
0.38231832
0.45105255
0.39380

KeyboardInterrupt: 